In [202]:
import os, sys
import urllib 
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
import json

endpoint_url = "https://query.wikidata.org/sparql"

img_data = {}

#Requete pour recuperer les infos de l'entité donnée - Q144/Chien par defaut
def query(wd='Q144', limit=5):
     return """SELECT ?item ?itemLabel ?pic{?item wdt:P31 wd:"""+wd+""".?item wdt:P18 ?pic}limit """+str(limit)+""""""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

#Pour creer le fichier des images s'il existe pas
def createFolder(name='images'):
    if name not in os.listdir():
        os.mkdir(name)
    else:
        print('Dossier '+name+' déjà exitant !')

#Pour verifier si un dossier est vide - evitons de retélécharger des images inutilement.
def isEmpty(name='images'):
    if len(os.listdir(name)) == 0:
        return True
    return False

#Pour ajouter des elements a la bdd
def addImg(wd,limit):
    results = get_results(endpoint_url, query(wd,limit))
    for result in results['results']['bindings']:
        img_data[result['item']['value'].split("/")[-1]] = {'image': result['pic']['value'],}
    #print(img_data)

def addAllImg(wd,limit):
    for w in wd:
        addImg(w,limit)

#Pour telecharger l'image d'un element de la bd
def downloadImage(url='https://upload.wikimedia.org/wikipedia/commons/a/ac/Bundesarchiv_B_145_Bild-F051673-0059%2C_Adolf_Hitler_und_Blondi_auf_dem_Berghof_%28cropped%29.jpg', name_img='img0'):
    full_path = 'images/'+name_img+'.jpg'
    urllib.request.urlretrieve(url, full_path)

#Si les images sont deja telechargées on initialise le tableau
def initAlreadyDownload(namefolder='images'):
    for image in os.listdir(namefolder):
        img_data[image] = {'image': ' ',}
    
#Pour télécharger toutes les images - limit par query
def downloadAllImages(wd=['Q144'], limit=5):
    createFolder()
    if isEmpty() == False:
        print('Images déjà téléchargées !')
        initAlreadyDownload()
        return False
    addAllImg(wd,limit)
    for i,img in enumerate(img_data):
        downloadImage(img_data[img]['image'], list(img_data.keys())[i])
    return True 

#Obtenir les metas d'une image
def getMetaImg(name):
    try:
        #print(name)
        imgfile = Image.open("images/"+name)
        #print(imgfile.size, imgfile.format)

        img_data[name]['size'] = imgfile.size
        img_data[name]['format'] = imgfile.format
        
        imgfile.close()
    except:
        removeImage(name)

        
#Obtenir les metas de toutes nos images    
def getMetaAllImg(namefolder='images'):
    for image in os.listdir(namefolder):
        getMetaImg(image)

#Supprimer une image du dossier 
def removeImage(image):
    print('Image '+image+' non valide, suppression de ses informations!')
    #os.remove('images/'+image)
    #img_data.pop(image)
    
def exportToJSON(name='img_data.json'):
    with open(name, 'w', encoding='utf-8') as f:
        json.dump(img_data, f, ensure_ascii=False, indent=4)


downloadAllImages(wd=['Q144','Q14660'],limit=10)
getMetaAllImg()
exportToJSON()


KeyError: 'Q86516809.jpg'